# Notebook for testing changes to vaccine data fetch script

In [ ]:
import pandas as pd
from datetime import date
import requests
from bs4 import BeautifulSoup
import json
import re

## Load new data from Texas DSHS

In [ ]:
d = date.today().strftime("%Y%m%d")
url = f"https://www.dshs.texas.gov/sites/default/files/LIDS-Immunize-COVID19/COVID%20Dashboard/County%20Dashboard/COVID-19%20Vaccine%20Data%20by%20County_{d}.xlsx"
# url = "../covid-texas-data/AccessibleVaccineDashboardData/2022-12-21.xlsx"
df = pd.read_excel(url, sheet_name="By County", index_col=[0], engine='openpyxl')

In [ ]:
df.drop('Federal Long-Term Care Vaccination Program', inplace=True) # Drop the federal data
df.drop('Federal Pharmacy Retail Vaccination Program', inplace=True)
df.drop('Other', inplace=True) # Drop the unknown county data
df.rename(index={"Texas": "Statewide"}, inplace=True)
df.index.names = ['county']

In [ ]:
df['date'] = pd.to_datetime(date.today())
df.set_index('date', append=True, inplace=True)

In [ ]:
df_new = df.loc[:, [
    'Vaccine Doses Administered', 
    'People Vaccinated with at least One Dose', 
    'People Fully Vaccinated',
    'People Vaccinated with at least One Booster Dose'
]]

In [ ]:
df_new.rename(columns={
    'Vaccine Doses Administered': 'total_doses',
    'People Vaccinated with at least One Dose': 'one_dose',
    'People Fully Vaccinated': 'vaccinated',
    'People Vaccinated with at least One Booster Dose': 'boosted'
}, inplace=True)

In [ ]:
df_new

### Update population counts

In [ ]:
df_pop = df.loc[:, ['Population 6Mo+']]
df_pop.rename(columns={'Population 6Mo+': 'population'}, inplace=True)
df_pop.reset_index(inplace=True)
df_pop.drop(columns='date', inplace=True)

In [ ]:
filename = "docs/vaccine/population.csv"
df_pop.to_csv(filename, float_format='%d', index=False)

### Load existing data and append today's data

In [ ]:
filename = "docs/vaccine/data.csv"
df = pd.read_csv(filename, index_col=['county', 'date'], parse_dates=['date'])

In [ ]:
df_merged = pd.concat([df, df_new])

In [ ]:
df_merged = df_merged[~df_merged.index.duplicated(keep='last')]
df_merged.sort_index(inplace=True)

In [ ]:
cols = df_merged.columns
df_merged[cols] = df_merged[cols].apply(pd.to_numeric)

In [ ]:
df_merged.to_csv(filename, float_format='%d', date_format='%Y-%m-%d')

## Alternative: Scrape data directly from Texas DSHS Vaccine Dashboard
Based on https://stackoverflow.com/a/62106733

In [ ]:
def getTableauSession(url):
    """Return Tableau session for use with sheet scraping"""
    r = requests.get(url, params={":embed": "y"})
    soup = BeautifulSoup(r.text, "html.parser")
    tableauData = json.loads(soup.find("textarea", {"id": "tsConfigContainer"}).text)
    dataUrl = f'https://tabexternal.dshs.texas.gov{tableauData["vizql_root"]}/bootstrapSession/sessions/{tableauData["sessionid"]}'
    r = requests.post(dataUrl, data={"sheet_id": tableauData["sheetId"]})
    return tableauData, r

def fetchSheet(tableauData, viewID):
    """Returns DataFrame of Tableau sheet with given view ID"""
    url = f"https://tabexternal.dshs.texas.gov{tableauData['vizql_root']}/vudcsv/sessions/{tableauData['sessionid']}/views/{viewID}?summary=true"
    return pd.read_csv(url)

In [ ]:
# Initiate Tableua session
url = "https://tabexternal.dshs.texas.gov/t/THD/views/COVID-19VaccineinTexasDashboard/Summary"
tableauData, r = getTableauSession(url)

In [ ]:
# Doses shipped (hardcoded into page)
state_shipped = re.findall("Doses Shipped  (\d{1,}(?:\,?\d{3})*)", r.text)
state_shipped = int(state_shipped[0].replace(",", ""))

In [ ]:
# Doses allocated
viewID = "2676828700685879255_608844250958173633"
sheet = fetchSheet(tableauData, viewID)
state_allocated = sheet.loc[0, "Measure Values"]

In [ ]:
# People vaccinated with at least one dose
viewID = "2676828700685879255_10709947349744759431"
sheet = fetchSheet(tableauData, viewID)
state_onedose = sheet.loc[0, "Measure Values"]

In [ ]:
# People fully vaccinated
viewID = "2676828700685879255_12258269643267636113"
sheet = fetchSheet(tableauData, viewID)
state_vaccinated = sheet.loc[0, "Measure Values"]

In [ ]:
# Doses administered
viewID = "2676828700685879255_15217363142870747880"
sheet = fetchSheet(tableauData, viewID)
state_doses = sheet.loc[0, "Measure Values"]
last_updated = pd.to_datetime(sheet.loc[0, "PublishData"])

In [ ]:
# County data (administered)
viewID = "2676828700685879255_10988292915152174895"
sheet = fetchSheet(tableauData, viewID)

sheet.drop(sheet.loc[sheet['County Name'] == 'Other'].index, inplace=True) # Drop the unknown county row
sheet['SumPeople'] = sheet['SumPeople'].apply(lambda x: pd.to_numeric(x.replace(",", "")))
sheet['SumPeopleFullyVax'] = sheet['SumPeopleFullyVax'].apply(lambda x: pd.to_numeric(x.replace(",", "")))

sheet_new = sheet[['County Name', 'SumPeople', 'SumPeopleFullyVax']].copy()
sheet_new.rename(columns={
    'County Name': 'county',
    'SumPeople': 'one_dose',
    'SumPeopleFullyVax': 'vaccinated',
}, inplace=True)

In [ ]:
sheet_new.insert(1, 'total_doses', sheet_new['one_dose'] + sheet_new['vaccinated'])
sheet_new.loc[-1] = ["Statewide", state_doses, state_onedose, state_vaccinated]

In [ ]:
sheet_new['date'] = pd.to_datetime(last_updated.normalize())
sheet_new.set_index(['county', 'date'], inplace=True)
sheet_new.sort_index(inplace=True)

In [ ]:
sheet_new

In [ ]:
df

## Load missing date from Texas Tribune data
From [rdmurphy/tx-covid-vaccine-data](https://github.com/rdmurphy/tx-covid-vaccine-data)

In [ ]:
date = '2021-03-24'

In [ ]:
df_new = pd.read_csv(f"https://raw.githubusercontent.com/rdmurphy/tx-covid-vaccine-data/main/distribution/snapshots/{date}.csv", index_col='County Name')
df_new.drop('Federal Long-Term Care Vaccination Program', inplace=True)
df_new.drop('Federal Pharmacy Retail Vaccination Program', inplace=True)
df_new.drop('Other', inplace=True)
df_new.rename(index={"Texas": "Statewide"}, inplace=True)
df_new.index.names = ['county']

In [ ]:
df_new['date'] = pd.to_datetime(date)
df_new.set_index('date', append=True, inplace=True)

In [ ]:
df_new = df_new.loc[:, [
    'Vaccine Doses Administered', 
    'People Vaccinated with at least One Dose', 
    'People Fully Vaccinated',
]]

In [ ]:
df_new.rename(columns={
    'Vaccine Doses Administered': 'total_doses',
    'People Vaccinated with at least One Dose': 'one_dose',
    'People Fully Vaccinated': 'vaccinated',
}, inplace=True)

In [ ]:
df_new

### Load existing data and merge

In [ ]:
filename = "docs/vaccine/data.csv"
df = pd.read_csv(filename, index_col=['county', 'date'], parse_dates=['date'])

In [ ]:
df_merged = pd.concat([df, df_new])

In [ ]:
df_merged = df_merged[~df_merged.index.duplicated(keep='last')]
df_merged.sort_index(inplace=True)

In [ ]:
df_merged

In [ ]:
df_merged.to_csv(filename, float_format='%d', date_format='%Y-%m-%d')